In [1]:
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image

In [2]:
Epochs = 100
Lr_Rate = 1e-3
Batch_Size = 128

### Make sure to resize the pixels array based upon the number of input images. here It is  given as 309 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
with open("numpy_data.npy","rb") as f:
    pixels = np.load(f)
    
pixels = np.resize(pixels,(309,1,128 ,128))

dataset = torch.from_numpy(pixels)

train_loader = data_utils.DataLoader(dataset, batch_size=Batch_Size, shuffle=True)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        #Encoder  16384
        self.enc1 = nn.Linear(in_features=16384, out_features=784) # Input image (128*128 = 16384)
        self.enc2 = nn.Linear(in_features=784, out_features=256) 
        self.enc3 = nn.Linear(in_features=256, out_features=128)
        self.enc4 = nn.Linear(in_features=128, out_features=64)
        self.enc5 = nn.Linear(in_features=64, out_features=32)
        self.enc6 = nn.Linear(in_features=32, out_features=16)

        #Decoder 
        self.dec1 = nn.Linear(in_features=16, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=128)
        self.dec4 = nn.Linear(in_features=128, out_features=256)
        self.dec5 = nn.Linear(in_features=256, out_features=784)
        self.dec6 = nn.Linear(in_features=784, out_features=16384) # Output image (128*128 = 16384)

    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))
        x = F.relu(self.enc6(x))

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        x = F.relu(self.dec6(x))

        return x

In [ ]:

def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

def make_dir():
    image_dir = 'Out_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
        
def save_decod_img(img, epoch):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, './Out_Images/Autoencoder_image{}.png'.format(epoch))
    
def training(model, train_loader, Epochs):
    train_loss = []
    for epoch in range(Epochs):
        running_loss = 0.0
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            img = img.view(img.size(0), -1)
            optimizer.zero_grad()
            outputs = model(img)
            loss = criterion(outputs, img)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        loss = running_loss / len(train_loader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(
            epoch+1, Epochs, loss))

        if epoch % 5 == 0:
            save_decod_img(outputs.cpu().data, epoch)

    return train_loss
        


In [ ]:
device = get_device()
model.to(device)
make_dir()
model = Autoencoder().to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=Lr_Rate)


In [ ]:
train_loss = training(model, train_loader, Epochs)

In [3]:
plt.figure()
plt.plot(train_loss)
plt.title('Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('deep_ae_mnist_loss.png')


16384

### Some of the generated images can be given here to create the reconstruction and based on that MSE can be calculated to check the novelty of our method

In [ ]:
decoded = model(image)
errors = []

# compute the mean squared error between the fake image
# and the reconstructed image, then add it to our list of errors
mse = np.mean((image - decoded) ** 2)
errors.append(mse)